# 💼 Customer Churn Analysis for Telecom
**Objective**: Predict and explain churn using EDA, ML, and SHAP  
**Tech Stack**: Python, LightGBM, SHAP, scikit-learn, pandas, seaborn  


## Step 1: Imports and Setup
This section includes all necessary libraries and global configurations.


In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve,ConfusionMatrixDisplay, precision_recall_curve, f1_score
import lightgbm as lgb


from scipy.stats import chi2_contingency
from statsmodels.stats.proportion import proportions_ztest
from itertools import combinations

import shap
import warnings

# --- Warnings Configuration ---
warnings.filterwarnings('ignore', category=FutureWarning, module='statsmodels')
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')


/Users/RICH/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/Users/RICH/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Step 2: Load Dataset
We'll use a Telco churn dataset containing customer demographics, service details, and churn labels.

In [2]:
df = pd.read_csv(r'/Users/RICH/Projects/Freelance_DS/Portofolio/churn_analysis_saas_t1/data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/RICH/Projects/Freelance_DS/Portofolio/churn_analysis_saas_t1/data/WA_Fn-UseC_-Telco-Customer-Churn.csv'

## Step 3: Clean and Prepare Data
We convert numeric fields, impute missing values, encode categorical variables, and drop the non-informative customer ID.

In [ ]:
# --- Step 3: Clean and Prepare Data ---

# 1. Fix data types
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)

# 2. Drop identifier column
df.drop('customerID', axis=1, inplace=True)

# 3. Map binary categorical columns safely (Yes/No → 1/0)
binary_cols = [
    'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'Churn'
]

for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].map({'Yes': 1, 'No': 0})

# 4. Double-check for remaining missing values
print("✅ Missing values:\n", df.isnull().sum())


## Step 4: Overall Churn Rate
We start by examining the proportion of customers who churned in the dataset.

In [ ]:
# --- Step 4: Overall Churn Rate ---
churn_rate = df['Churn'].mean()
print(f"Overall Churn Rate: {churn_rate:.2%}")


## Step 5: Exploratory Churn Analysis

We analyze how customer churn relates to both categorical and continuous features.

- **Categorical Variables**: Contract type, Internet service, Payment method, Senior citizen status, Online security, and Tech support.
- **Continuous Variable**: Tenure.

These insights help highlight which customer segments are at higher risk of churning.


In [ ]:
# --- Step 5: Exploratory Churn Analysis by Category and Numeric Features ---

# Categorical features to plot
categorical_cols = [
    'Contract', 'InternetService', 'PaymentMethod', 
    'OnlineSecurity', 'TechSupport', 'SeniorCitizen'
]

# Reusable plot function for categorical churn breakdown
def plot_churn_by_category(col):
    plt.figure(figsize=(8, 5))
    sns.countplot(x=col, hue='Churn', data=df)
    plt.title(f'Churn by {col}')
    plt.xlabel(col)
    plt.ylabel('Number of Customers')
    plt.legend(title='Churn', labels=['No', 'Yes'])
    plt.xticks(rotation=30)
    plt.tight_layout()
    plt.show()

# Generate plots for all selected categorical columns
for col in categorical_cols:
    plot_churn_by_category(col)

# Plot for tenure (continuous variable)
plt.figure(figsize=(8, 6))
sns.histplot(data=df, x='tenure', hue='Churn', multiple='stack', bins=30)
plt.title('Churn by Tenure')
plt.xlabel('Tenure (Months)')
plt.ylabel('Number of Customers')
plt.tight_layout()
plt.show()


## Step 6: Correlation Analysis

We compute the correlation matrix for all numeric features. This helps:
- Detect multicollinearity
- Understand how strongly churn correlates with other variables


In [ ]:
# --- Step 6: Correlation Analysis for Numeric Features ---

# Select only numeric features
numeric_features = df.select_dtypes(include=[np.number])

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(numeric_features.corr(), annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Correlation Heatmap of Numeric Features')
plt.tight_layout()
plt.show()


## Step 7: Statistical Significance Testing

We perform:

- **Chi-Square tests** to evaluate whether churn is associated with each categorical feature.
- **Pairwise Z-tests** to assess whether specific levels (e.g., “Fiber optic” vs. “DSL”) differ significantly in churn.

Results are visualized and saved to CSV.


In [ ]:
# --- Step 7: Chi-Square & Z-Test for Categorical Features ---

from itertools import combinations
from statsmodels.stats.proportion import proportions_ztest

alpha = 0.05  # significance level
results = []

# Ensure churn is numeric
if df['Churn'].dtype == object or df['Churn'].isna().all():
    df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

# List of categorical features to test
categorical_features = [
    'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
    'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
    'PaperlessBilling', 'PaymentMethod', 'Contract'
]

for col in categorical_features:
    print(f"\n===== Feature: {col} =====")

    # Encode values to ensure consistency
    encoded_col = f"{col}_encoded"
    df[encoded_col] = LabelEncoder().fit_transform(df[col].astype(str))
    mapping = dict(zip(df[encoded_col], df[col].astype(str)))

    # Chi-square test
    ct = pd.crosstab(df[encoded_col], df['Churn'])
    if ct.shape[0] < 2 or ct.shape[1] < 2 or (ct.sum(axis=1) == 0).any():
        print(f"Skipping chi2 for {col}: not enough variation")
        continue

    chi2, p_chi2, dof, expected = chi2_contingency(ct)
    print(f"Chi2 = {chi2:.2f}, p = {p_chi2:.4f} -> {'Significant' if p_chi2 < alpha else 'Not Significant'}")

    # Barplot
    plt.figure(figsize=(8, 4))
    sns.barplot(x=col, y='Churn', data=df, ci=None)
    plt.title(f'Churn Rate by {col}')
    plt.ylabel('Churn Rate')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Pairwise Z-tests
    print("Pairwise Z-tests:")
    for g1, g2 in combinations(df[encoded_col].unique(), 2):
        d1 = df[df[encoded_col] == g1]['Churn']
        d2 = df[df[encoded_col] == g2]['Churn']
        if min(d1.count(), d2.count()) == 0:
            continue
        count = [d1.sum(), d2.sum()]
        nobs = [d1.count(), d2.count()]
        stat, pval = proportions_ztest(count, nobs)
        result = "Significant" if pval < alpha else "Not Significant"
        print(f"  {mapping[g1]} vs {mapping[g2]}: p = {pval:.4f} => {result}")

        results.append({
            'Feature': col,
            'Group 1': mapping[g1],
            'Group 2': mapping[g2],
            'p-value': pval,
            'Result': result
        })

# Save pairwise test results
pd.DataFrame(results).to_csv("pairwise_churn_significance_tests.csv", index=False)



## Key Findings from EDA and Statistical Testing

- The overall churn rate in the dataset is **~26%**.
- **Contract Type** has a significant relationship with churn (p < 0.001).
  - Customers on **Month-to-Month** contracts churn at a much higher rate than those on **One-Year** or **Two-Year** contracts.
- **Paperless Billing** is associated with higher churn.
- **Online Security** and **Tech Support** are strong differentiators:
  - Customers **without Online Security** or **without Tech Support** churn at much higher rates.
- **Senior Citizens** are slightly more likely to churn compared to non-seniors, though the effect is moderate.
- **Payment Method** shows significant differences:
  - Customers using **Electronic Check** have a much higher churn rate than those using **Bank Transfers** or **Credit Cards**.

### Summary of Significant Features:

| Feature            | Significant Relationship with Churn? |
|--------------------|-------------------------------------|
| Contract           | Yes                                 |
| Paperless Billing  | Yes                                 |
| Online Security    | Yes                                 |
| Tech Support       | Yes                                 |
| Payment Method     | Yes                                 |
| Senior Citizen     | Moderate                            |
| Multiple Lines     | Slight                              |
| Internet Service   | Yes                                 |

---

### **Recommendations Based on EDA**
- Encourage customers to switch to **longer-term contracts** (One-Year, Two-Year).
- Promote **bundled services** like Online Security and Tech Support to reduce churn.
- Target customers with **Electronic Check** payment method for intervention.

                                                        

## Step 8: Feature Selection & Preprocessing

Based on our EDA and statistical tests, we selected features most relevant to churn. In this step:

- We removed irrelevant or leakage columns (e.g., `customerID`, temp encoded features).
- Mapped binary categorical values (`Yes`/`No`) to 1/0 for modeling.
- Applied one-hot encoding to multi-class categorical features (e.g., `Contract`, `PaymentMethod`) to convert them into numeric format.
- Ensured numeric columns (`TotalCharges`, `MonthlyCharges`, `tenure`) were properly converted and imputed where necessary.
- Replaced infinite values and filled any remaining missing values with column medians.

At this point, all features are numeric and ready for model training.


In [ ]:
# Step 7: Feature Selection and Preprocessing

# 1. Copy dataset and drop unnecessary columns
df_model = df.copy()
df_model.drop(columns=[col for col in ['customerID'] if col in df_model.columns], inplace=True)
df_model.drop(columns=[col for col in df_model.columns if col.endswith('_encoded')], inplace=True)

# 2. Define target and selected features based on EDA
target = 'Churn'
selected_features = [
    'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService',
    'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
    'Contract', 'PaperlessBilling', 'PaymentMethod', 
    'MonthlyCharges', 'TotalCharges'
]

X = df_model[selected_features].copy()
y = df_model[target]

# 3. Binary encoding for Yes/No columns
binary_cols = [
    'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]
for col in binary_cols:
    if col in X.columns:
        X[col] = X[col].apply(lambda x: 1 if x == 'Yes' else 0)

# 4. One-hot encode multi-category columns
multi_cat_cols = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod']
X = pd.get_dummies(X, columns=[col for col in multi_cat_cols if col in X.columns], drop_first=True)

# 5. Ensure all numeric columns are valid and clean
for col in ['TotalCharges', 'MonthlyCharges', 'tenure']:
    X[col] = pd.to_numeric(X[col], errors='coerce')
    X[col].fillna(X[col].median(), inplace=True)

# 6. Final pass for infs or NaNs
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(X.median(), inplace=True)

# 7. Sanity check
print("✅ Remaining NaNs:", X.isnull().sum().sum())  # Should be 0
print("📊 Data Types:\n", X.dtypes.value_counts())  # Should show int64 and float64


## 🔢 Step 9: Modeling and Evaluation

We trained and evaluated three different models to predict customer churn:

1. **Logistic Regression**
2. **Random Forest**
3. **LightGBM**

Each model was evaluated using standard classification metrics and ROC/Precision-Recall analysis. For Random Forest and LightGBM, we also tuned the classification threshold to maximize the F1-score.

---

### 📌 Logistic Regression

- **Model trained** with scaled features (standardization).
- Evaluated on default threshold = 0.5.
- Key Metrics:
  - ROC AUC: High
  - Precision: High (identifies churners with confidence)
  - Recall: Moderate (misses some churners)

---

### 🌲 Random Forest

- **Trained without scaling** (tree-based models don't require it).
- Threshold tuned to **0.24** for optimal F1-score.
- Key Metrics:
  - Improved Recall compared to Logistic Regression.
  - Slight drop in Precision.
- Included Precision-Recall vs Threshold plot and Confusion Matrix at selected threshold.

---

### 💡 LightGBM

- **Gradient Boosting Model**, efficient and fast.
- Threshold tuned to **0.23** to balance precision and recall.
- Delivered the **best F1-score** among all models.
- Feature importance chart highlights key drivers of churn.

---

### 🧪 Evaluation Criteria

For all models, we used:

- **Classification Report** (Precision, Recall, F1)
- **Confusion Matrix**
- **ROC Curve**
- **Precision-Recall Curve**
- **Custom threshold tuning** to maximize F1
- **Feature Importance** (coefficients or impurity gain)

Each model was compared in a summary table to aid final selection.


In [ ]:
## Train a logistic regression model on the scaled training data
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Logistic Regression Model
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)

# Make Predictions
y_pred_log = log_model.predict(X_test_scaled)
y_proba_log = log_model.predict_proba(X_test_scaled)[:, 1]

# Evaluate performance using classification report, confusion matrix, and ROC AUC
print("📋 Classification Report:")
print(classification_report(y_test, y_pred_log))

conf_matrix = confusion_matrix(y_test, y_pred_log)
print("🧮 Confusion Matrix:")
print(conf_matrix)

#Visualize confusion matrix

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

# ROC AUC Score
roc_auc = roc_auc_score(y_test, y_proba_log)
print(f"📈 ROC AUC Score: {roc_auc:.4f}")

# ROC Curve Plot
fpr, tpr, thresholds = roc_curve(y_test, y_proba_log)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')  # random guess
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# --- Feature Importance: Logistic Regression ---

# Extract feature names and coefficients
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': log_model.coef_[0]
})

# Add absolute coefficient for sorting
feature_importance['Abs_Coefficient'] = feature_importance['Coefficient'].abs()
feature_importance = feature_importance.sort_values(by='Abs_Coefficient', ascending=False)

# Plot
plt.figure(figsize=(12, 8))
sns.barplot(
    y='Feature', x='Coefficient', 
    data=feature_importance,
    palette='coolwarm'
)
plt.title('Feature Importance (Logistic Regression)')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


In [ ]:
# --- Step 9.2: Random Forest Model ---

# Train a Random Forest model (doesn't require feature scaling)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predictions
y_pred_rf = rf_model.predict(X_test)
y_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Evaluate its performance using classification metrics and ROC AUC
print("Random Forest Classification Report:")
print(classification_report(y_test, y_pred_rf))

roc_auc_rf = roc_auc_score(y_test, y_proba_rf)
print(f"ROC AUC Score: {roc_auc_rf:.4f}")

# Plot ROC curve for visual comparison
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)

plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_rf:.2f})', color='darkorange')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Random Forest')
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()


In [ ]:
# Plot precision and recall across probability thresholds
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_rf)

plt.figure(figsize=(8, 6))
plt.plot(thresholds, precision[:-1], label='Precision', linestyle='--')
plt.plot(thresholds, recall[:-1], label='Recall')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision-Recall vs Threshold (Random Forest)')
plt.legend(loc='best')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Initialize best values
best_threshold = 0
best_f1 = 0

# Find the threshold that gives the best F1 score (balance between precision and recall)
for thresh in thresholds:
    y_pred_thresh = (y_proba_rf >= thresh).astype(int)
    f1 = f1_score(y_test, y_pred_thresh)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = thresh

print(f"Best Threshold: {best_threshold:.2f} with F1 Score: {best_f1:.4f}")


In [ ]:
# Apply optimal threshold (from F1 tuning) to predicted probabilities
optimal_threshold = 0.24
y_pred_opt = (y_proba_rf >= optimal_threshold).astype(int)

# Print classification report to evaluate performance metrics at new threshold
print(f"Random Forest Classification Report (Threshold {optimal_threshold:.2f}):")
print(classification_report(y_test, y_pred_opt))

# Visualize the confusion matrix for better interpretation
cm_opt = confusion_matrix(y_test, y_pred_opt)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_opt, display_labels=["No Churn", "Churn"])
disp.plot(cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix at Threshold {optimal_threshold:.2f}')
plt.show()

# AUC is threshold-independent and remains constant
print(f"Random Forest ROC AUC Score: {roc_auc_rf:.4f}")


In [ ]:

# 1. Initialize LightGBM Model
lgb_model = lgb.LGBMClassifier(random_state=42, n_estimators=100)

# 2. Fit the Model
lgb_model.fit(X_train, y_train)

# 3. Predict Probabilities
y_proba_lgb = lgb_model.predict_proba(X_test)[:, 1]

# 4. ROC AUC Score
roc_auc_lgb = roc_auc_score(y_test, y_proba_lgb)
print(f"LightGBM ROC AUC Score: {roc_auc_lgb:.4f}")

# 5. Precision-Recall Curve
precision_lgb, recall_lgb, thresholds_lgb = precision_recall_curve(y_test, y_proba_lgb)

# 6. Find Best Threshold (Maximize F1 Score)
best_thresh_lgb = 0
best_f1_lgb = 0

for thresh in thresholds_lgb:
    preds_thresh = (y_proba_lgb >= thresh).astype(int)
    f1 = f1_score(y_test, preds_thresh)
    if f1 > best_f1_lgb:
        best_f1_lgb = f1
        best_thresh_lgb = thresh

print(f"Best Threshold for LightGBM: {best_thresh_lgb:.2f} with F1 Score: {best_f1_lgb:.4f}")

# 7. Classification Report at Best Threshold
y_pred_lgb_best = (y_proba_lgb >= best_thresh_lgb).astype(int)
print("LightGBM Classification Report (Best Threshold):")
print(classification_report(y_test, y_pred_lgb_best))

# 8. Confusion Matrix at Best Threshold
from sklearn.metrics import ConfusionMatrixDisplay
cm_lgb = confusion_matrix(y_test, y_pred_lgb_best)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_lgb, display_labels=["No Churn", "Churn"])
disp.plot(cmap=plt.cm.Blues)
plt.title(f'Confusion Matrix at Threshold {best_thresh_lgb:.2f}')
plt.show()

# 9. Precision-Recall vs Threshold Plot
plt.figure(figsize=(8,6))
plt.plot(thresholds_lgb, precision_lgb[:-1], label='Precision')
plt.plot(thresholds_lgb, recall_lgb[:-1], label='Recall')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision-Recall vs Threshold (LightGBM)')
plt.legend()
plt.show()

# 10. Feature Importance Plot
importances = lgb_model.feature_importances_
features = X.columns

feat_imp = pd.Series(importances, index=features).sort_values(ascending=False)

plt.figure(figsize=(12, 8))
feat_imp[:20].plot(kind='barh')
plt.gca().invert_yaxis()
plt.title('Feature Importance (LightGBM)')
plt.xlabel('Importance Score')
plt.show()


## 🔚 Final Model Comparison Summary

We evaluated three models — **Logistic Regression**, **Random Forest**, and **LightGBM** — for customer churn prediction, comparing performance using ROC AUC, Accuracy, Precision, Recall, and F1-Score. Thresholds for Random Forest and LightGBM were chosen to maximize the F1-Score.

### 📊 Performance Metrics

| Metric                     | Logistic Regression (0.50) | Random Forest (0.24) | LightGBM (0.23) |
|---------------------------|----------------------------|----------------------|-----------------|
| ROC AUC                   | 0.8431                     | 0.8145               | 0.8292          |
| Accuracy                  | 0.80                       | 0.73                 | 0.74            |
| Precision (Churners)      | 0.64                       | 0.49                 | 0.50            |
| Recall (Churners)         | 0.53                       | 0.76                 | 0.81            |
| F1-Score (Churners)       | 0.58                       | 0.60                 | 0.62            |

### 📝 Summary

- **Logistic Regression** achieved the **highest ROC AUC (0.8431)** and **precision (64%)**, but with **lower recall (53%)**, meaning it missed many true churners.
- **Random Forest**, tuned at a **0.24 threshold**, improved recall to **76%**, though with reduced precision (**49%**).
- **LightGBM**, tuned at a **0.23 threshold**, offered the **best balance**, with **81% recall** and **50% precision**, leading to the **highest F1-Score (62%)**.

### ✅ Final Model Selection

We recommend **LightGBM** as the final model due to its **superior ability to identify churners**, balancing the business need for **high recall** while maintaining **acceptable precision**.


## Step 10: Customer Scoring & Risk Segmentation

Using the final LightGBM model, we scored all customers with a predicted churn probability and segmented them into actionable risk groups.

### 🔢 Scoring
- Each customer received a **churn probability** based on the LightGBM model.

### 🟡 Risk Segmentation
- **High Risk**: Churn probability > 0.6  
- **Medium Risk**: 0.3 < Probability ≤ 0.6  
- **Low Risk**: Probability ≤ 0.3  

This segmentation enables focused retention strategies and helps prioritize outreach efforts.

### 📊 Segment Distribution
We analyzed the proportion of customers in each segment to understand the size of the potential churn base.

---



In [ ]:
#Score All Customers Using Final Model

# Predict churn probabilities using LightGBM
customer_churn_scores = lgb_model.predict_proba(X)[:, 1]  # Probability of churn (class 1)

# Create a new DataFrame with scores
scored_customers = X.copy()
scored_customers['Churn_Probability'] = customer_churn_scores

# Preview top rows
scored_customers[['Churn_Probability']].head()


In [ ]:
#Risk Segmentation

# Define risk bands based on churn probability
def risk_segment(prob):
    if prob > 0.6:
        return 'High Risk'
    elif prob > 0.3:
        return 'Medium Risk'
    else:
        return 'Low Risk'

# Apply segmentation
scored_customers['Risk_Segment'] = scored_customers['Churn_Probability'].apply(risk_segment)

# Display segment distribution
risk_counts = scored_customers['Risk_Segment'].value_counts()
print("📊 Risk Segment Distribution:")
print(risk_counts)


In [ ]:
# Feature Importance (LightGBM)

# Extract feature importances
importances = lgb_model.feature_importances_
features = X.columns
feat_imp = pd.Series(importances, index=features).sort_values(ascending=False)

# Plot top 20 features
plt.figure(figsize=(12, 8))
feat_imp[:20].plot(kind='barh')
plt.gca().invert_yaxis()
plt.title('Top 20 Feature Importances (LightGBM)')
plt.xlabel('Importance Score')
plt.tight_layout()
plt.show()

# Generate short summary of top 5 features
top_features = feat_imp[:5].index.tolist()

feature_summary_text = f"""
Top indicators of customer churn based on the LightGBM model are:
1. {top_features[0]}
2. {top_features[1]}
3. {top_features[2]}
4. {top_features[3]}
5. {top_features[4]}

These features are the most influential in predicting customer churn and should be prioritized for business interventions.
"""
print(feature_summary_text)



## Step 11: Model Explainability with SHAP

To increase trust and interpretability of the model, SHAP (SHapley Additive exPlanations) was used to explain both **global** and **individual** churn predictions.

### 🌍 Global Feature Importance
- A **summary plot** shows which features most strongly affect predictions across the dataset.
- Confirms that features like `tenure`, `Contract_Two year`, and `OnlineSecurity` have the greatest global impact.

### 👤 Individual Force Plots
- For the top high-risk customers, **SHAP force plots** highlight how specific features pushed their churn risk up or down.
- These can support **personalized retention** campaigns by revealing the unique churn factors for each customer.

SHAP enhances model transparency, aligns predictions with business logic, and empowers human-in-the-loop decisions.

In [ ]:
# SHAP Explanation (Global Feature Impact)

# 1. Initialize SHAP explainer using the trained LightGBM model
explainer = shap.TreeExplainer(lgb_model)

# 2. Compute SHAP values for the test set (binary classification: returns list of arrays)
shap_values = explainer.shap_values(X_test)

# 3. Plot global feature importance (mean absolute SHAP values)
shap.summary_plot(shap_values, X_test, plot_type="bar")


In [ ]:
# SHAP Explanation (Local Interpretation for Top High-Risk Test Customers)

# Identify top N high-risk customers in the test set
top_n = 5
high_risk_test = X_test.copy()
high_risk_test['Churn_Probability'] = y_proba_lgb
high_risk_test_sorted = high_risk_test.sort_values(by='Churn_Probability', ascending=False).head(top_n)

# Loop through top N customers and show force plots
shap.initjs()

for i, idx in enumerate(high_risk_test_sorted.index):
    print(f"\nCustomer {i+1} - Index {idx}")
    display(
        shap.force_plot(
            explainer.expected_value[1],          # Expected value for churn class
            shap_values[1][X_test.index.get_loc(idx)],  # Correct index inside X_test
            X_test.loc[idx]                       # Feature values for that customer
        )
    )


In [ ]:
# Initialize JS for SHAP interactive plots
shap.initjs()

# Force plot for a specific customer (interactive in notebook)
customer_idx = 0  # Change index as needed
shap.force_plot(
    explainer.expected_value[1],
    shap_values[1][customer_idx],
    X_test.iloc[customer_idx]
)


## 🧩 Steps 10 & 11 Summary – Scoring, Segmentation & Model Explainability

### 🎯 Customer Scoring & Segmentation (Step 10)

Using the final LightGBM model, each customer was scored with a predicted probability of churn. Based on this score, customers were segmented into:

- **High Risk**: Probability > 60%
- **Medium Risk**: 30–60%
- **Low Risk**: ≤ 30%

This segmentation enables targeted retention strategies by identifying which customers require immediate attention and which are more stable.

### 🔍 Explainability with SHAP (Step 11)

To interpret and validate model behavior, SHAP was used at both **global** and **individual** levels:

- **Global SHAP Summary**:
  - Key churn drivers included: `Contract_2`, `tenure`, `InternetService_1`, `MonthlyCharges`, and `Contract_1`.
  - Features associated with short customer lifespan, high cost, and no commitment had the strongest impact on churn predictions.

- **Individual SHAP Force Plots** (Top 5 Risk Cases):
  - Nearly all top churners were:
    - On **month-to-month contracts**
    - Had **low tenure**
    - Were **senior citizens** with **high monthly charges**
    - Subscribed to **fiber optic internet** without clear added value
  - These drivers appeared repeatedly, making them strong targets for churn intervention efforts.

SHAP added transparency to the model’s decision-making and allowed for **human-aligned, customer-specific action plans**.


# 🧾 Final Project Summary – Telecom Churn Prediction

This end-to-end analysis focused on understanding and predicting customer churn for a telecom business using a mix of exploratory analysis, statistical testing, and machine learning.

---

## 📌 Key Phases

1. **Exploratory Data Analysis & Statistical Testing**
   - Identified churn rate (~26%)
   - Found that churn was significantly related to contract type, support services, and billing method
   - Statistical z-tests and chi-square confirmed visual insights

2. **Modeling & Evaluation**
   - Compared Logistic Regression, Random Forest, and LightGBM
   - LightGBM tuned to threshold 0.23 achieved:
     - **ROC AUC**: 0.8292
     - **Recall (churners)**: 81%
     - **Precision (churners)**: 50%
     - **F1 Score**: **62%** (best among all models)

3. **Customer Scoring & Segmentation**
   - Customers scored on churn probability
   - Segmented into high, medium, and low risk

4. **Model Explainability (SHAP)**
   - Global SHAP plots validated important churn drivers
   - Force plots for top 5 high-risk customers revealed shared risk patterns:
     - Short tenure
     - Month-to-month contract
     - High monthly charges
     - Fiber optic internet
     - Senior citizen status

---

## ✅ Recommendations

- Prioritize **contract upgrades** (to one- or two-year terms)
- Offer **loyalty incentives** to **high-paying, low-tenure** customers
- Bundle or promote **Online Security and Tech Support**
- Focus retention campaigns on **High Risk** segment identified by LightGBM scoring

---

## 🚀 Final Outcome

LightGBM was selected as the best model for production due to its balance of performance and interpretability. The project delivered a fully interpretable churn scoring system, actionable segmentation, and visual insights to inform business strategy.


In [ ]:
rename_dict = {
    'SeniorCitizen': 'Is Senior Citizen',
    'Partner': 'Has Partner',
    'Dependents': 'Has Dependents',
    'tenure': 'Tenure (Months)',
    'PhoneService': 'Has Phone Service',
    'OnlineSecurity': 'Has Online Security',
    'OnlineBackup': 'Has Online Backup',
    'DeviceProtection': 'Has Device Protection',
    'TechSupport': 'Has Tech Support',
    'StreamingTV': 'Has Streaming TV',
    'StreamingMovies': 'Has Streaming Movies',
    'PaperlessBilling': 'Uses Paperless Billing',
    'MonthlyCharges': 'Monthly Charges',
    'TotalCharges': 'Total Charges',
    
    'MultipleLines_No phone service': 'No Phone Service',
    'MultipleLines_Yes': 'Has Multiple Lines',
    
    'InternetService_Fiber optic': 'Fiber Internet',
    'InternetService_No': 'No Internet Service',
    
    'Contract_One year': 'One-Year Contract',
    'Contract_Two year': 'Two-Year Contract',
    
    'PaymentMethod_Credit card (automatic)': 'Pays by Credit Card (Auto)',
    'PaymentMethod_Electronic check': 'Pays by Electronic Check',
    'PaymentMethod_Mailed check': 'Pays by Mailed Check',
    
    'Churn_Probability': 'Predicted Churn Probability',
    'Risk_Segment': 'Risk Segment'
}


In [ ]:
scored_customers_readable = scored_customers.rename(columns=rename_dict)

In [ ]:
print(scored_customers_readable.columns.tolist())

In [ ]:
scored_customers_readable.to_csv("churn_scored_customers.csv", index=False)

In [ ]:
scored_customers_readable